## <span style="color:#ff5f27">📝 Imports </span>

In [ ]:
import datetime
from features.price import generate_historical_data, to_wide_format, plot_historical_id

import great_expectations as ge
from great_expectations.core import ExpectationSuite, ExpectationConfiguration

## <span style="color:#ff5f27">⚙️ Data Generation </span>

In [ ]:
data_generated = generate_historical_data(
    datetime.date(2022, 9, 1),
)
data_generated.head(3)

In [ ]:
plot_historical_id([1,2], data_generated)

## <span style="color:#ff5f27"> 👮🏻‍♂️ Great Expectations </span>

In [ ]:
ge_price_df = ge.from_pandas(data_generated)
expectation_suite_price = ge_price_df.get_expectation_suite()
expectation_suite_price.expectation_suite_name = "price_suite"

In [ ]:
expectation_suite_price.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "id",
            "min_value": 0,
            "max_value": 1000,
        }
    )
)

expectation_suite_price.add_expectation(
    ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "price",
            "min_value": 0,
            "max_value": 1000,
        }
    )
)

for column in ['date', 'id', 'price']:
    expectation_suite_price.add_expectation(
        ExpectationConfiguration(
            expectation_type="expect_column_values_to_be_null",
            kwargs={
                "column": column,
                "mostly": 0.0,
            }
        )
    )

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

## <span style="color:#ff5f27">🪄 Feature Group Creation </span>

In [ ]:
prices_fg = fs.get_or_create_feature_group(
    name='prices',
    description='Prices Data',
    version=1,
    primary_key=['id'],
    event_time='date',
    online_enabled=True,
    expectation_suite=expectation_suite_price,
)    
prices_fg.insert(data_generated)

## <span style="color:#ff5f27"> 👩🏻‍🔬 Data Transformation to Wide Format </span>

In [ ]:
prices_fg = fs.get_or_create_feature_group(
    name='prices',
    version=1,
)
prices_df = prices_fg.read()
prices_df.head(5)

In [ ]:
prices_df_wide = to_wide_format(prices_df)
prices_df_wide.head(3)

In [ ]:
prices_df_wide.isna().sum()[prices_df_wide.isna().sum() > 0]

---